# Players account info handler

In [1]:
import time

from itertools import chain

import requests as req
import pandas as pd
import numpy as np

from settings import TOKEN

## Goal
Retrieve the account_id from the players account info becuase with it we can start retrieving the match history of each player

## 1. Import players pool data

In [2]:
players_pool = pd.read_pickle("raw_data/players_pool.pkl")
players_pool.head()

,tier,rank,summonerId,summonerName,leaguePoints,wins,losses
0,DIAMOND,I,rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp,Meiran,90,200,187
1,DIAMOND,I,Q1MxBiPaWxpvLUOcHTyUTGIRLoOsQ0pPW2GXJpFzlShKg4J_,Preciöus,0,101,79
2,DIAMOND,I,UHd2YrByzhM458uxwF_sR5saxARizi-quOa-S30OmFH41Or8,ˆˇˇˆˇˇˆ,52,160,146
3,DIAMOND,I,JbkMW-xXj99fD7CesxP-4Q7HTbTVw_DjEAXhMuPMGqj1xEZt,Huskigodx,3,264,261
4,DIAMOND,I,2RiE8ql63iwtSSYQYA_f84PtrCX_js2FrAWlOJfI4Wm83uES,Ghiosu,30,448,434


In [3]:
id_list = players_pool["summonerId"].to_list()
id_list[0]

'rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp'

## 2. Check how the api works

In [4]:
def get_acc_info(summoner_id: str):
    
    url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={TOKEN}"
    res = req.get(url)
    
    return res.json()

In [5]:
get_acc_info(id_list[0])

{'id': 'rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp',
 'accountId': 'Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg',
 'puuid': '7d4EK5NVYbakMY9VXZ6DSUWMkTp_eHYL3phhtSVPR5OkgqQBM0NrMM8iQej9Ka2fSDjRMR_4X3kggQ',
 'name': 'Meiran',
 'profileIconId': 4568,
 'revisionDate': 1594471446000,
 'summonerLevel': 229}

- We are interested on storing the accountId value into the players data

# 3. Adjust the function to retrieve each individual account information

- max 100 request every 2 minutes

In [12]:
def get_accounts_info(id_list: list, token: str, path: str, start=0, end=0) -> pd.DataFrame:
    container = []
    base_url = "https://euw1.api.riotgames.com/lol/summoner/v4/summoners"
    n_requests = 0
    while start < end:
        # request account info for each summoner_id
        for i in range(start, end):
            # control the number of requests
            if n_requests >= 100:
                print(f"Number of Entries so far: {len(container)}")
                n_requests = 0
                time.sleep(121)

            # make request
            url = f"{base_url}/{id_list[i]}?api_key={token}"
            res = req.get(url)
            if res.status_code == 200:
                container.append(res.json())
                # update position
                start += 1
            else:
                # if error save to csv
                pd.DataFrame(container).to_csv(
                    f"{path}/fragmented_data/account_info_{start}.pkl", index=False)
                print(f"Something went wrong {res.status_code}")
                break
            # update the number of requests
            n_requests += 1
    print(f"Done, number of elements: {len(container)}")
    # return useful data
    container = pd.DataFrame(container).rename(columns={"id": "summonerId"})
    container = container[["summonerId", "accountId"]]
    container.to_pickle(f"{path}/account_info.pkl", protocol=4)
    return f"File saved at {path}/account_info.pkl"

- Execute the function and hope for the best :D

In [13]:
get_accounts_info(id_list, TOKEN, "raw_data", 0, 100)

Done, number of elements: 100


'File saved at raw_data/account_info.pkl'

## 4. Check the data (already done in the function above this is just a step by step)

In [35]:
first = pd.DataFrame(first)
print(f"Columns: {first.shape[1]} Rows: {first.shape[0]}\n")
print(f"Number of duplcated rows: {first.duplicated().sum()}\n")
print("Null values per column:\n")
print(first.isna().sum())

Columns: 7 Rows: 200

Number of duplcated rows: 0

Null values per column:

id               0
accountId        0
puuid            0
name             0
profileIconId    0
revisionDate     0
summonerLevel    0
dtype: int64


- Rename the id column to match summonerId on the player's pool

In [39]:
first = first.rename(columns={"id": "summonerId"})

- Select the keys we want

In [41]:
first = first[["summonerId", "accountId"]]

## 5. Merge the data from accounts to player_pool data

In [44]:
players_pool = players_pool.merge(first, on="summonerId")

In [48]:
players_pool.head()

,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,accountId
0,DIAMOND,I,rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp,Meiran,90,200,187,Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg
1,DIAMOND,I,Q1MxBiPaWxpvLUOcHTyUTGIRLoOsQ0pPW2GXJpFzlShKg4J_,Preciöus,0,101,79,2O_DVP7Wz4HWs039wGEKgdNsSrt_E3mQv9SZnEEH-G2LYt4
2,DIAMOND,I,UHd2YrByzhM458uxwF_sR5saxARizi-quOa-S30OmFH41Or8,ˆˇˇˆˇˇˆ,52,160,146,D3JuXBnzCjgcpOe3l-mfjOFv6_CypuLMKonCRtFWxxhYJl0
3,DIAMOND,I,JbkMW-xXj99fD7CesxP-4Q7HTbTVw_DjEAXhMuPMGqj1xEZt,Huskigodx,3,264,261,K1II1_OHR-OOdOvq_pqmDLqCJh7UH7A7RijWxD1erWu7KxY
4,DIAMOND,I,2RiE8ql63iwtSSYQYA_f84PtrCX_js2FrAWlOJfI4Wm83uES,Ghiosu,30,448,434,6-ZpLloiZHOa7RjJSXDYC9N_835Hi6uIlzAe2fU5T7H4kIg


## 6. Export the data

In [45]:
players_pool.to_pickle("raw_data/players_pool_acc_id.pkl", protocol=4)

# As a function

In [15]:
def merge_with(players_pool: pd.DataFrame, container: pd.DataFrame, path: str):
    players_pool = players_pool.merge(container, on="summonerId")
    players_pool.to_pickle(f"{path}/players_pool_account.pkl", protocol=4)
    return f"File saved at {path}/players_pool_account.pkl"

In [18]:
acc_info = pd.read_pickle("raw_data/account_info.pkl")
acc_info.head()

,summonerId,accountId
0,rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp,Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg
1,Q1MxBiPaWxpvLUOcHTyUTGIRLoOsQ0pPW2GXJpFzlShKg4J_,2O_DVP7Wz4HWs039wGEKgdNsSrt_E3mQv9SZnEEH-G2LYt4
2,UHd2YrByzhM458uxwF_sR5saxARizi-quOa-S30OmFH41Or8,D3JuXBnzCjgcpOe3l-mfjOFv6_CypuLMKonCRtFWxxhYJl0
3,JbkMW-xXj99fD7CesxP-4Q7HTbTVw_DjEAXhMuPMGqj1xEZt,K1II1_OHR-OOdOvq_pqmDLqCJh7UH7A7RijWxD1erWu7KxY
4,2RiE8ql63iwtSSYQYA_f84PtrCX_js2FrAWlOJfI4Wm83uES,6-ZpLloiZHOa7RjJSXDYC9N_835Hi6uIlzAe2fU5T7H4kIg


In [19]:
merge_with(players_pool, acc_info, "raw_data")

'File saved at raw_data/players_pool_account.pkl'

- Check the file

In [20]:
merged = pd.read_pickle("raw_data/players_pool_account.pkl")
merged.head()

,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,accountId
0,DIAMOND,I,rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp,Meiran,90,200,187,Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg
1,DIAMOND,I,Q1MxBiPaWxpvLUOcHTyUTGIRLoOsQ0pPW2GXJpFzlShKg4J_,Preciöus,0,101,79,2O_DVP7Wz4HWs039wGEKgdNsSrt_E3mQv9SZnEEH-G2LYt4
2,DIAMOND,I,UHd2YrByzhM458uxwF_sR5saxARizi-quOa-S30OmFH41Or8,ˆˇˇˆˇˇˆ,52,160,146,D3JuXBnzCjgcpOe3l-mfjOFv6_CypuLMKonCRtFWxxhYJl0
3,DIAMOND,I,JbkMW-xXj99fD7CesxP-4Q7HTbTVw_DjEAXhMuPMGqj1xEZt,Huskigodx,3,264,261,K1II1_OHR-OOdOvq_pqmDLqCJh7UH7A7RijWxD1erWu7KxY
4,DIAMOND,I,2RiE8ql63iwtSSYQYA_f84PtrCX_js2FrAWlOJfI4Wm83uES,Ghiosu,30,448,434,6-ZpLloiZHOa7RjJSXDYC9N_835Hi6uIlzAe2fU5T7H4kIg
